In [1]:
import pandas as pd
import numpy as np
import os
import tqdm
import shutil

In [19]:
dataset = 'emma' # onion or emma
results_dir = f'../results/' + dataset + '/performance/'
cutoffs = [1, 5, 10, 20]
cutoff_date = '2025_03_13'
metrics = ['Recall', 'Precision', 'nDCG', 'MRR', 'HR', 'EFD', 'EPC', 'ARP', 'PopREO', 'PopRSP', 'ItemCoverage', 'UserCoverage']
#metrics = ['Recall', 'Precision', 'nDCG', 'MRR', 'HR']

In [20]:
def find_modality(string):
    keyword = "modalites="
    start = string.find(keyword)
    if start == -1:
        return None

    start += len(keyword)
    end = string.find("_", start)
    if end == -1:
        end = len(string)

    modality = string[start:end].strip("'")
    return modality

In [21]:
# read all files in result folder
files = os.listdir(results_dir)
df = pd.DataFrame()

for cutoff in cutoffs:
    df_cut = pd.DataFrame()
    result_files_cutoff = [f for f in files if f.startswith(f'rec_cutoff_{cutoff}_')]
    for f in result_files_cutoff:
        df_tmp = pd.read_csv(results_dir + f, sep='\t')
        tmp_metrics = [m for m in metrics if m in df_tmp.columns]
        df_tmp.rename(columns={m: f'{m}@{cutoff}' for m in tmp_metrics}, inplace=True)
        df_tmp['modality'] = find_modality(df_tmp['model'].values[0])
        df_tmp['model'] = df_tmp['model'].str.split('_').str[0]
        df_tmp['date'] = f[-23:-4]
        df_cut = pd.concat([df_cut, df_tmp], axis=0)

    # keep only newest model results
    df_cut = df_cut.sort_values('date').drop_duplicates(subset=['model', 'modality'], keep='last')

    # drop models results before cutoff date
    df_cut = df_cut[df_cut['date'] >= cutoff_date]

    # merge with previous cutoffs
    if not df.empty:
        df = pd.merge(df, df_cut, on=['model', 'date', 'modality'], how='inner')
        print(df.shape)
    else:
        df = df_cut

df = df[['model', 'modality', 'date'] + [c for c in df.columns if '@' in c]]
df.sort_values('nDCG@5', inplace=True)
# display columns with @10 only
#df[['model', 'modality', 'date'] + [c for c in df.columns if '@1' in c]]
df[['model', 'modality', 'date'] + ['Recall@1', 'Recall@5', 'Recall@10', 'nDCG@1', 'nDCG@5', 'nDCG@10']]

(7, 27)
(7, 39)
(7, 51)


,model,modality,date,Recall@1,Recall@5,Recall@10,nDCG@1,nDCG@5,nDCG@10
4,LightGCNM,emotion,2025_03_13_21_45_13,0.122674,0.358082,0.499281,0.220869,0.287274,0.341346
6,LightGCNM,audio'-'textual'-'visual'-'emotion,2025_03_14_05_01_08,0.127858,0.367327,0.511901,0.229078,0.294675,0.350147
1,LightGCN,None,2025_03_13_15_10_00,0.132696,0.369212,0.517846,0.237023,0.298911,0.355824
2,FeatureItemKNN,emotion,2025_03_13_17_14_05,0.135741,0.374032,0.511244,0.242585,0.303772,0.356463
3,FeatureItemKNN,audio'-'textual'-'visual,2025_03_13_21_09_45,0.141435,0.375215,0.514565,0.252383,0.307470,0.361218
5,FeatureItemKNN,audio'-'textual'-'visual'-'emotion,2025_03_13_22_11_58,0.141435,0.375215,0.514565,0.252383,0.307470,0.361218
0,ItemKNN,None,2025_03_13_13_24_47,0.140998,0.374729,0.515979,0.251324,0.307664,0.362160


In [13]:
# export to comma seperated file
df.to_csv(f'../results/{dataset}_performance.csv', index=False)

In [14]:
for cutoff in cutoffs:
    df_cut = df[['model'] + [c for c in df.columns if f'@{cutoff}' in c]]
    df_cut.to_csv(f'../results/{dataset}_performance_{cutoff}.csv', index=False)